# How proximity to volcano hazards affects housing value, income, and demographic distribution of people in Big Island, Hawaii.

For this project, I will be working with census data of Hawaii and other GIS data to make an interactive map that shows the distribution of housing value and demographic income data in relation to the proximity of volcanoes on the big island in Hawaii. This juypter notebook consists of using various python packages to add interactivity to my final project. In addition, there are maps that uses census block data along with interactivity functions to let people explore census data.  

In [ ]:
! pip install leafmap
import fiona
import os
import numpy as np
import json
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider, ColorPicker, jslink, IntRangeSlider
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from ipyleaflet import Map, basemaps, basemap_to_tiles, MagnifyingGlass, MeasureControl, GeoData, LayersControl, DrawControl, WidgetControl,LegendControl, SearchControl, Marker, AwesomeIcon
from ipyleaflet import *
import itertools 
import ipyleaflet
from branca.colormap import linear
import leafmap # This is a super cool package!!
import leafmap.colormaps as cm

# First, Lets see what an eruption looks like!!


Do you see the volcano fire in the SouthEast part of the island??

In [ ]:
# comparsion map of what the eruption look like
m = Map(center=(19.441755, -155.444437), zoom=9) # Making a map

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2020-12-23") # Modis satellite true color bands
left_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisAquaBands721CR, "2020-12-23") # Modis satellite aqua bands

marker1 = Marker(name='marker1', location=(19.44,-155.30))  # Change the marker style
m.add_layer(marker1)

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer) # Adding split map control function to the map
m.add_control(control)

m

# Let's check out the lava hazard index on Big Island.

In [ ]:
#lava hazard
hazard_file = gpd.read_file("lava_hazard.shp") # read lava hazard shapefile and assign to a variable
hazard_map= hazard_file.explore(column="SEVERITYCO", name = "Severity",cmap = "Reds") # Set the parameter
folium.LayerControl().add_to(hazard_map) # Add map control to the map
hazard_map

# Adding interactivity to display blocks by income level


In [ ]:
# Define a feature that reads shapefiles
def readfeatures(shpfilename): 
    fh = fiona.open(shpfilename, 'r')
    featurelist = [] # create an empty list
    for feature in fh: # loop over features in shapefile
        featurelist.append(feature) # append each feature to the empty list
    fh.close() # close the shapefile
    return featurelist

#returns a basic folium map
def createbasicmap(zoom_level = 6):
    m = folium.Map(location=[19.741755, -155.844437], zoom_start=zoom_level) # Create a folium map
    return m

# Add shapefile feature to the map
def addfeaturetomap(feature,folium_map):
    j = json.dumps(feature)  # dumps the features
    folium.features.GeoJson(j).add_to(folium_map) # add the features to the map

    
# Define a function that only shows block groups that are fallen within a income range

my_map = createbasicmap() # assign a variable to the basic map function
feature_list = readfeatures("Median_Income.shp") # assign variable to the shapefile

def map_some_tracts(income):
    my_map = createbasicmap() # create basic map
    
    for feature in feature_list: # loop over feature in feature list 
        if feature['properties']['JOIE001'] <= income: # set conditionals to make the interactivev slider only show income within certain values
            addfeaturetomap(feature, my_map) # add conditional features to the map
    return my_map

interact(map_some_tracts, income= 50000) # function call

# Let's explore Hawaii!

In [ ]:
m = Map(center=(19.94,-155.04), zoom = 7, basemap= basemaps.Esri.WorldTopoMap) # create a map with parameters
demo_maps = gpd.read_file("Demographics.shp") # assign variable 
geo_data = GeoData(geo_dataframe = demo_maps,
                   style={'color': 'red', 'fillColor': '#0000FF', 'opacity':0.1, 'weight':0.1, 'dashArray':'9', 'fillOpacity':0.3},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Blocks') # set the parameters

m.add_layer(geo_data) # add the geodata to the map
m

# Add Measure control


In [ ]:
m = Map(center=(19.94,-155.04), zoom = 7, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = demo_maps, # set the parameters for the map
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.1, 'weight':1.5, 'dashArray':'2', 'fillOpacity':0.7},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.4},
                   name = 'Blocks')

measure = MeasureControl(
    position='topright',
    active_color = 'red',
    primary_length_unit = 'miles'
) # set parameters for the measure contorl 

measure.completed_color = 'red'

measure.add_length_unit('yards', 1.09361, 4)
measure.secondary_length_unit = 'yards'

measure.add_area_unit('sqyards', 1.19599, 4)
measure.secondary_area_unit = 'sqyards'

# add the interactivities to the map
m.add_control(measure)
m.add_layer(geo_data)
m.add_control(LayersControl())

m

# Add Drawing function!

In [ ]:
m = Map(center=(19.94,-155.04), zoom = 7, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = demo_maps,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Blocks')

# Add polyline
draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "#A52A2A",
        "weight": 9,
        "opacity": 1.2
    }
}
# Add polygon
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#0000FF",
        "color": "#6be5c3",
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#FF0000",
        "message": "wth!"
    },
    "allowIntersection": False
}
# Add circle
draw_control.circle = {
    "shapeOptions": {
        "fillColor": "#800000",
        "color": "#7FFD4",
        "fillOpacity": 1.0
    }
}
# Add rectangle
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

m.add_control(draw_control)
m.add_layer(geo_data)

m

# Add widget control

In [ ]:
m = Map(center=(19.94,-155.04), zoom = 7, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = demo_maps,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Blocks')

# Add zoom slider function
zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=7)
jslink((zoom_slider, 'value'), (m, 'zoom'))
widget_control1 = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widget_control1)

# Add color picker function
color_picker = ColorPicker(description='Pick a color:')
widget_control2 = WidgetControl(widget=color_picker, position='bottomright')

m.add_control(widget_control2)
m.add_layer(geo_data)

m

# Let's add a legend!

In [ ]:
m = Map(center=(19.94,-155.04), zoom=7)

legend = LegendControl({"low":"#FAA", "medium":"#A55", "High":"#500"}, name="Legend", position="bottomright")
m.add_control(legend)


# Set legend title
legend.name = "Legend"  # Set name
legend.name  # Get name

# Set legend content
legend.legends = {"Land":"#FFF", "Mountain":"#10F92F", "Ocean":"#10ABF9"}  # Set content
legend.legends  # Get content

# legend position
legend.position = "topright"  # Set position
legend.position  # Get current position

m

# How about a search function!!

In [ ]:
m = Map(zoom=7, center=[19.94,-155.04])

# Add marker for the location you are trying to serach
marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

# Add search control function
m.add_control(SearchControl(
  position="topleft",
  url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
  zoom=5,
  marker=marker
))

m

# Add all the things together on a single map!

In [ ]:
m = Map(center=(19.94,-155.04), zoom = 7, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = demo_maps,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Blocks')
measure = MeasureControl(
    position='bottomright',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
)


measure.completed_color = 'red'

measure.add_length_unit('yards', 1.09361, 4)
measure.secondary_length_unit = 'yards'

measure.add_area_unit('sqyards', 1.19599, 4)
measure.secondary_area_unit = 'sqyards'

draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "#6bc2e5",
        "weight": 8,
        "opacity": 1.0
    }
}
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}
draw_control.circle = {
    "shapeOptions": {
        "fillColor": "#efed69",
        "color": "#efed69",
        "fillOpacity": 1.0
    }
}
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=7)
jslink((zoom_slider, 'value'), (m, 'zoom'))
widget_control1 = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widget_control1)

color_picker = ColorPicker(description='Pick a color:')
widget_control2 = WidgetControl(widget=color_picker, position='bottomright')

legend = LegendControl({"low":"#FAA", "medium":"#A55", "High":"#500"}, name="Legend", position="bottomright")
m.add_control(legend)



# Set/Get legend title
legend.name = "Legend"  # Set name
legend.name  # Get name

# Set/Get legend content
legend.legends = {"Land":"#FFF", "Mountain":"#10F92F", "Ocean":"#10ABF9"}  # Set content
legend.legends  # Get content


# legend position
legend.position = "topright"  # Set position
legend.position  # Get current position

marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

m.add_control(SearchControl(
  position="topright",
  url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
  zoom=5,
  marker=marker
))

m.add_control(measure)
m.add_control(widget_control2)
m.add_control(draw_control)
m.add_control(LayersControl())
m.add_layer(geo_data)
m

# Let's explore some census data!

# But first, lets clean some useless columns!

In [ ]:
medianincome = gpd.read_file("Median_Income.shp") # Read the median income shapefile
# Drop useless columns
medianincome1 = medianincome.drop(columns=['STATEFP','COUNTYFP','TRACTCE','BLKGRPCE','AFFGEOID','GEOID','NAME','LSAD','ALAND','AWATER','Shape_Leng','Shape_Area','Geoid_dbl','GISJOIN','STATEA','COUNTYA','TRACTA','BLKGRPA','NAME_E','NAME_M','JOIM001','GEOID_1'])
medianincome1.rename(columns={"JOIE001":"Median_Income"}, inplace=True) # Rename column
medianincome1

In [ ]:
# Map it--Median Income
medianmap = medianincome1.explore(column="Median_Income", name = "Median Income", scheme='naturalbreaks') # display map with desired column from the shapefile
folium.LayerControl().add_to(medianmap) # Add layercontrol to the map
medianmap


# Clean columns for demographic shapefile

In [ ]:
demographic = gpd.read_file("Race.shp") # Read race shapefile
# Drop useless columns
demographic1 = demographic.drop(columns=['STATEFP','COUNTYFP','TRACTCE','BLKGRPCE','AFFGEOID','GEOID','NAME','LSAD','ALAND','AWATER','Shape_Leng','Shape_Area','geoid_dbl','GISJOIN','YEAR','REGIONA','DIVISIONA','STATEA','COUNTYA','Field9','TRACTA','BLKGRPA','NAME_1','SABINSA','Geoid_1'])
demographic1.rename(columns={"American_I":"American_Indian", "Native_Haw":"Native_Hawaiian", "H7X001":"Total_Population"}, inplace=True) # rename some columns
demographic1

# Make sure to deselect all the layers in the layer control in the map below first before checking each race!! Otherwise, they will stack all together and the choropleth map will not show the layers correctly.

In [ ]:
# Demographic

demographicmap = folium.Map(location=[19.5666644,-155.499998], zoom_start=7) # Set some basic parameters for map

explore_columns = ["Asian","White","Black","American_Indian","Native_Hawaiian", "Other_Race"] # create a list for the columns that I'm going to show in the map
for col_str in explore_columns:  # Using for loop to show layers
    demographic1.explore(m=demographicmap, column=col_str, name=col_str, scheme='naturalbreaks') # display desired columns

folium.LayerControl().add_to(demographicmap) # Add layer control to the map
demographicmap

# Notice the layer control has differen layer selection option!
# Also, running this code cell is going to take a while because of the 'natural breaks' scheme...

# Clean data for house value

In [ ]:
house_value = gpd.read_file("house_value.shp") # Read house value shapefile
# Drop useless columns
house_value1 = house_value.drop(columns=['STATEFP','COUNTYFP','TRACTCE','BLKGRPCE','AFFGEOID','GEOID','NAME','LSAD','ALAND','AWATER','Shape_Leng','Shape_Area','GISJOIN','STATEA','COUNTYA','TRACTA','BLKGRPA','Geoid_dbl','Field2','Geoid_1'])
house_value1.rename(columns={"JTIE001":"House_value"}, inplace=True) # rename some columns
house_value1


In [ ]:
# Map house value
housevaluemap = house_value1.explore(column="House_value", name = "Median House Value", scheme = "NaturalBreaks") # Display desired columns
folium.LayerControl().add_to(housevaluemap) # Add layer control to the map
housevaluemap

# Now, let's find the relationship between lava hazard and income, demographics, and race data!!

# First, lets find the top 25% house value in Hawaii

In [ ]:
house_value1['House_value'].plot(kind='box') # Plot the boxplot to see the value
house_value1.House_value.quantile([0.25,0.5,0.75]) # Get the precise value of top 25%


It seems like the top 25% is about 654100 dollars.

# Plot the rich blocks in Hawaii County (big island)

In [ ]:
# house_value1.House_value.idxmax()
# richest =house_value1['House_value'][59] # Assign richest as a variable to the house_value of the richest blocks
rich_blocks = house_value1[(house_value1.House_value >654100) & (house_value1.COUNTY=='Hawaii County')].plot(figsize = (15,15)) # Plot the top 25% area


# Lets see them in a map!

In [ ]:
rich_blocks1 = house_value1[(house_value1.House_value >654100) & (house_value1.COUNTY=='Hawaii County')] # make rich block only in Hwaii county and has a house value that is greater than top 25%
rich_blocks_map = rich_blocks1.explore(column="House_value", name = "Median House Value", scheme='naturalbreaks') # Display desired column
folium.LayerControl().add_to(rich_blocks_map) # Add layer control to the map
rich_blocks_map

# Overlay Function-- to see the relations between the two layers

In [ ]:
Lava_hazard = gpd.read_file("lava_hazard.shp") # Read lava hazard shapefile
overlay = gpd.overlay(rich_blocks1, Lava_hazard, 'union') # using overlay function
overlaymap = folium.Map(location=[19.5666644,-155.499998], zoom_start=7) # Assign variable to create a map
explore_columns1 = ["House_value","SEVERITYCO"]
for col_str1 in explore_columns1: # Using for loops to explore the overlay layers
    overlay.explore(m=overlaymap, column=col_str1, scheme='naturalbreaks')

folium.LayerControl().add_to(overlaymap)    
overlaymap

# Now, let's see the relationship between income and lava hazard!

First, lets find top 30% income people in Hawaii.

In [ ]:
medianincome1['Median_Income'].plot(kind='box') # Plot the boxplot to see the value
medianincome1.Median_Income.quantile([0.25,0.5,0.70]) # Get the precise value of top 25%


Plot the rich blocks in Hawaii County (big island)

In [ ]:
income_blocks = medianincome1[(medianincome1.Median_Income >= 83028) & (medianincome1.COUNTY=='Hawaii County')].plot(figsize = (15,15)) # Plot the top 25% area

In [ ]:
rich_blocks2 = medianincome1[(medianincome1.Median_Income > 83028) & (medianincome1.COUNTY=='Hawaii County')] # make rich block only in Hwaii county and has a house value that is greater than top 25%
rich_blocks_map1 = rich_blocks2.explore(column="Median_Income", name = "Median Income", scheme='naturalbreaks') # Assign variable to display desired column on a map
folium.LayerControl().add_to(rich_blocks_map1)
rich_blocks_map1

# Overlay median income and lava hazard shapefile to see if there is any correlation 

In [ ]:
overlay1 = gpd.overlay(rich_blocks2, Lava_hazard, 'union') # using overlay function
overlaymap1 = folium.Map(location=[19.5666644,-155.499998], zoom_start=7) # Assign variable to display map

explore_columns2 = ["SEVERITYCO","Median_Income"]
for col_str2 in explore_columns2: # Using for loops to explore the overlay layers
    overlay1.explore(m=overlaymap1, column=col_str2, scheme='naturalbreaks') # display columns from the loop
    
folium.LayerControl().add_to(overlaymap1)   # Add layer control to the map
overlaymap1

# Lastly, let's explore demographic relationship with lava hazard!

First, gotta do some data manipulation.

In [ ]:
demographic1['percent_color'] = (demographic1["Asian"] + demographic1["Black"] + demographic1["American_Indian"] + demographic1["Native_Hawaiian"] + demographic1["Other_Race"]) / demographic1["Total_Population"] # Get the percentage of population of people of color
demographic1.fillna(0) # fill the NaN value to 0

Now, display the people of color as percentage to the total population on a map.

In [ ]:
big_island_ppl = demographic1[(demographic1.COUNTY=='Hawaii County')] # Make a new list that has only Hawaii County demographic data
ppl_color_map = big_island_ppl.explore(column="percent_color", name = "People of color", scheme='naturalbreaks') # Display the desired column in a map
ppl_color_map

Now overlay lava hazard and demographic layer

In [ ]:
overlay2 = gpd.overlay(big_island_ppl, Lava_hazard, 'union') # using overlay function
overlaymap2 = folium.Map(location=[19.5666644,-155.499998], zoom_start=7) # Craete a map

explore_columns3 = ["SEVERITYCO","percent_color"]
for col_str3 in explore_columns3: # Using for loops to explore the overlay layers
    overlay2.explore(m=overlaymap2, column=col_str3, scheme='naturalbreaks') # Display the desired column on a map
    
folium.LayerControl().add_to(overlaymap2)   # Add layer control to my map
overlaymap2